# Running Semantic Functions Inline

The [previous notebook](./02-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your C# code. This can be useful in a few scenarios:

* Dynamically generating the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating. 

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the kernel arguments.

Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

In [4]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#!import config/Settings.cs
#!import config/BuilderForOllama.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;

var kernel = BuilderForOllama.BuildKernel("deepseek-r1");


Installed Packages Microsoft.SemanticKernel, 1.23.0

Using model: deepseek-r1:1.5b, ollama, 


Let's create a semantic function used to summarize content:

In [5]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [ ]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000, //최대 토큰 수 
    Temperature = 0.2, //창의성 수준
    TopP = 0.5 //샘플링 0~1
    //FrequencyPenalty = 0.0 //반복 단어 패널티 -2~2
    //PresncePenalty = 0.0 //새로운 단어 유도 -2~2
    //StopSequences = new List<string> { "###" } //"###" 키워드에서 멈춤
    //ModelId = "deepseek-r1" //사용할 모델 ID
    //SeviceId = "default-openai" //여러 AI서비스 중 선택
    //NumberOfRetries = 3 //실패 시 재시도 횟수
};

# Semantic Kernel OpenAIPromptExecutionSettings 설정 옵션

`OpenAIPromptExecutionSettings`는 OpenAI API 호출 시 AI 모델의 동작을 제어하는 다양한 매개변수를 설정할 수 있는 클래스입니다. Ollama 환경에서도 OpenAI 호환 API를 사용하므로 동일하게 적용됩니다.

### 주요 설정 옵션 상세 설명

#### 1. **MaxTokens** (최대 토큰 수)
응답의 최대 길이를 제한합니다.
- **범위**: 1 ~ 모델의 최대값
- **기본값**: 모델에 따라 다름 (보통 4096)
- **설명**: 토큰은 단어의 일부 또는 전체를 나타냄. 1000 토큰 ≈ 750단어

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    MaxTokens = 2000  // 최대 2000 토큰까지 응답 가능
};
````

#### 2. **Temperature** (창의성 수준)
응답의 일관성과 창의성을 조절합니다.
- **범위**: 0 ~ 2
- **권장값**: 0 ~ 1 (2 이상은 불안정할 수 있음)
- **0에 가까울수록**: 결정적, 일관된 답변
- **1에 가까울수록**: 창의적, 다양한 답변

````csharp
// 보수적인 답변 (예: 사실 요약)
var conservativeSettings = new OpenAIPromptExecutionSettings 
{ 
    Temperature = 0.2 
};

// 창의적인 답변 (예: 시 작성, 아이디어 브레인스토밍)
var creativeSettings = new OpenAIPromptExecutionSettings 
{ 
    Temperature = 0.9 
};
````

#### 3. **TopP** (누클러스 샘플링)
확률 상위 P% 범위의 단어만 선택하여 다양성과 품질의 균형을 맞춥니다.
- **범위**: 0 ~ 1
- **권장**: Temperature와 함께 사용하되, 하나만 조정하는 것이 권장됨
- **0.9**: 상위 90%의 선택지 중에서 선택
- **0.5**: 상위 50%의 선택지만 고려 (더 일관된 답변)

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    TopP = 0.5  // 가장 가능성 높은 50%의 단어들만 고려
};
````

#### 4. **FrequencyPenalty** (반복 페널티)
같은 단어가 반복되는 것을 억제합니다.
- **범위**: -2 ~ 2
- **0**: 페널티 없음
- **양수**: 반복된 단어 억제
- **음수**: 반복된 단어 장려 (권장하지 않음)

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    FrequencyPenalty = 0.5  // 같은 단어 반복 억제
};
````

#### 5. **PresencePenalty** (신규 단어 장려)
새로운 주제나 단어 등장을 장려합니다.
- **범위**: -2 ~ 2
- **0**: 페널티 없음
- **양수**: 새로운 주제 장려
- **음수**: 같은 주제 반복 (권장하지 않음)

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    PresencePenalty = 0.5  // 새로운 주제 등장 장려
};
````

#### 6. **StopSequences** (정지 시퀀스)
특정 문자열이 나타나면 생성을 멈추도록 합니다.
- **최대 4개** 시퀀스 설정 가능
- **용도**: 불필요한 생성 방지, 형식 제어

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    StopSequences = new List<string> 
    { 
        "\n\n",      // 두 줄 띄어쓰기에서 멈춤
        "###",       // "###"에서 멈춤
        "END"        // "END" 문자열에서 멈춤
    }
};
````

#### 7. **ModelId** (모델 선택)
사용할 모델을 명시적으로 지정합니다.

````csharp
// OpenAI의 경우
var settings = new OpenAIPromptExecutionSettings 
{ 
    ModelId = "gpt-4"  // 또는 "gpt-3.5-turbo"
};

// Ollama의 경우
var settings = new OpenAIPromptExecutionSettings 
{ 
    ModelId = "deepseek-r1"
};
````

#### 8. **ServiceId** (서비스 선택)
여러 AI 서비스가 등록되었을 때 특정 서비스를 지정합니다.

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    ServiceId = "primary-openai"  // 또는 "backup-azure-openai"
};
````

#### 9. **NumberOfRetries** (재시도 횟수)
API 호출 실패 시 재시도하는 횟수입니다.

````csharp
var settings = new OpenAIPromptExecutionSettings 
{ 
    NumberOfRetries = 3
};
````

### 실제 사용 예제

#### 요약 작업 (정확성 우선)
````csharp
var summarySettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 500,        // 짧은 요약
    Temperature = 0.2,      // 낮은 창의성
    TopP = 0.5,             // 선택지 제한
    FrequencyPenalty = 0.0  // 반복 허용
};

var summaryFunction = kernel.CreateFunctionFromPrompt(
    """
    {{$input}}
    
    위 내용을 3줄로 요약해주세요.
    """, 
    summarySettings
);
````

#### 창의적 콘텐츠 생성 (다양성 우선)
````csharp
var creativeSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,            // 충분한 길이
    Temperature = 1.0,           // 높은 창의성
    TopP = 0.95,                 // 다양한 선택
    FrequencyPenalty = 0.5,      // 반복 억제
    PresencePenalty = 0.5        // 새로운 아이디어 장려
};

var jokeFunction = kernel.CreateFunctionFromPrompt(
    """
    {{$input}}에 대한 재미있는 농담을 만들어주세요.
    """, 
    creativeSettings
);
````

#### 코드 생성 (정확성과 일관성)
````csharp
var codeSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 1500,
    Temperature = 0.2,          // 정확한 코드
    StopSequences = new List<string> { "```", "\n\n\n" }
};

var codeFunction = kernel.CreateFunctionFromPrompt(
    """
    {{$input}}에 대한 C# 코드를 작성하세요.
    """, 
    codeSettings
);
````

### Ollama 환경에서의 주의사항

Ollama를 사용할 때는 모델의 성능에 따라 설정을 조정해야 합니다:

````csharp
// Ollama 소형 모델용 설정 (deepseek-r1:1.5b 등)
var ollamaSmallSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 1000,          // 더 작은 크기
    Temperature = 0.3,         // 낮은 창의성
    TopP = 0.7,
    FrequencyPenalty = 0.3
};

// Ollama 대형 모델용 설정
var ollamaLargeSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.9
};
````

### 설정 비교 표

| 사용 사례 | Temperature | TopP | MaxTokens | FrequencyPenalty | 목적 |
|---------|-----------|------|-----------|------------------|------|
| 요약 | 0.2 | 0.5 | 500 | 0.0 | 정확성 |
| 질문 답변 | 0.3 | 0.7 | 1000 | 0.2 | 신뢰성 |
| 창의 글쓰기 | 0.9 | 0.95 | 2000 | 0.5 | 다양성 |
| 코드 생성 | 0.2 | 0.5 | 1500 | 0.0 | 정확성 |
| 브레인스토밍 | 1.0 | 0.95 | 1500 | 0.5 | 창의성 |

### 추가 자료

- [OpenAI API 매개변수 공식 문서](https://platform.openai.com/docs/api-reference/chat/create)
- [Semantic Kernel OpenAI 설정 가이드](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/openai/)
- [Temperature와 Top P 이해하기](https://platform.openai.com/docs/guides/gpt-best-practices)
- [Ollama 모델 최적화 가이드](https://github.com/ollama/ollama/blob/main/docs/modelfile.md)
- [Semantic Kernel 공식 예제](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples)

The following code prepares an instance of the template, passing in the TXT and configuration above, 
and a couple of other parameters (how to render the TXT and how the template can access other functions).

This allows to see the prompt before it's sent to AI.

In [7]:
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

Console.WriteLine(renderedPrompt);



Summarize the content above.


Let's transform the prompt template into a function that the kernel can execute:

In [8]:
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Set up some content to summarize, here's an extract about Demo, an ancient Greek poet, taken from [Wikipedia](https://en.wikipedia.org/wiki/Demo_(ancient_Greek_poet)).

In [9]:
var input = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
""";

...and run the summary function:

In [ ]:
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });

Console.WriteLine(summaryResult);

The code above shows all the steps, to understand how the function is composed step by step. However, the kernel
includes also some helpers to achieve the same more concisely.

The same function above can be executed with less code:

In [ ]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);

Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text.




In [ ]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = textToSummarize });

Console.WriteLine(result);